# Test the differential layer and the Heisenber layer with coherent states

Creat a multimode coherent state and test the output of the DifferentialGaussianLayer

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>


nonlinearxwaves@gmail.com<br>
@version 5 january 2021<be>
@version 23 sep 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as kb

In [3]:
tf.keras.backend.clear_session()

In [4]:
 #np.set_printoptions(precision=2)

## Dimension (number of modes times 2, N=2n)

In [5]:
N = 4

## Build vacuum by the Gaussian state

In [6]:
vacuum = ps.VacuumLayer(N)

## Coherent state (Displacer on mode 0 and 1)

In [7]:
A_np = 10.0
lambda_np = np.pi/2
dinput=np.zeros((N,1));
dinput[2]=np.sqrt(2)*A_np*np.cos(lambda_np);
dinput[3]=np.sqrt(2)*A_np*np.sin(lambda_np);
displacer=ps.DisplacementLayerConstant(dinput)

Alpha parameters on the modes

In [8]:
alpha=np.zeros((2,),dtype=complex)
alpha[0]=complex(dinput[0]/np.sqrt(2), dinput[1]/np.sqrt(2))
alpha[1]=complex(dinput[2]/np.sqrt(2), dinput[3]/np.sqrt(2))
print(alpha)

[0.000000e+00 +0.j 6.123234e-16+10.j]


### Dummy training points (not used for training in this example)

In [9]:
Nbatch=10
gtarget=np.eye(N)
dtarget=2.4*np.ones((N,1))
xtrain = np.random.rand(Nbatch, N)-0.5
ytrain =np.zeros_like(xtrain)
dtrain = np.zeros((Nbatch,N))
gtrain = np.zeros((Nbatch,N,N))
for j in range(Nbatch):
    for i in range(N):
        dtrain[j,i]=dtarget[i]
        for k in range(N):
            gtrain[j,i,k]=gtarget[i,k]

# Build the model

In [10]:
xin = tf.keras.layers.Input(N)
x0, a0 = displacer(xin)
chir, chii = vacuum(x0, a0)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])

# Evaluate the uncertainties by DifferentialGaussianLayer 

# Build the tensor with the covariance matrix and 

### evaluate the covariance and displacement of the model

In [11]:
g, d, hessian = ps.CovarianceLayer(N)(chir, chii, model)
covarianceModel = tf.keras.Model(inputs = xin, outputs=[g, d])
print(covarianceModel(xtrain))

[<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 1., -0., -0., -0.],
       [-0.,  1., -0., -0.],
       [-0., -0.,  1., -0.],
       [-0., -0., -0.,  1.]], dtype=float32)>, <tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[0.0000000e+00, 0.0000000e+00, 8.6595603e-16, 1.4142136e+01]],
      dtype=float32)>]


### build the tensors

In [12]:
gdiag = tf.linalg.diag_part(g)
d2 = tf.square(d)
d4 = tf.square(d2)

# build tensor gjj gkk
gjjkk = tf.tensordot(gdiag,gdiag, axes=0)

# build tensor gjj dk dk
gjjdkdk = tf.tensordot(gdiag, d2, axes =0)


In [13]:
print(gdiag)
print(gjjkk)

KerasTensor(type_spec=TensorSpec(shape=(4,), dtype=tf.float32, name=None), name='tf.linalg.diag_part/diag_part:0', description="created by layer 'tf.linalg.diag_part'")
KerasTensor(type_spec=TensorSpec(shape=(4, 4), dtype=tf.float32, name=None), name='tf.tensordot/Tensordot/MatMul:0', description="created by layer 'tf.tensordot'")


# Compare with the differential layer

In [14]:
Diff=ps.DifferentialGaussianLayer(N)
nboson, Dn, Dn2 = Diff(chir,chii, model)
HModel = tf.keras.Model(inputs = xin, outputs=[nboson, Dn, Dn2])

# Run the model

In [15]:
nboson1, Dn, Dn2=HModel(xtrain) 

# Compare the number of bosons with expected values

For coherent state <n> = |alpha|^2

Computed model

In [16]:
tf.print(nboson1)

[[0 100]]


Theoretically expected value

In [17]:
n0ex=np.abs(alpha)**2
print(np.abs(alpha)**2)

[  0. 100.]


# Compare the < nj-nk >

In [18]:
tf.print(Dn)

[[0 -100]
 [100 0]]


# Compare the < (nj-nk)^2 > 

For the coherent state 
< (n1-n0)^2 >= <n1^2>+<n0^2>-2<n0> <n1>=|alpha0|^4+|alpha0|^2+|alpha1|^4+|alpha1|^2-2 |alpha1|^1 |alpha0|^2

In [19]:
expected_value = n0ex[0]**2+n0ex[0]+n0ex[1]**2+n0ex[1]-2*n0ex[0]*n0ex[1]

In [20]:
print(expected_value)

10100.0


Computed value

In [21]:
tf.print(Dn2)

[[0 10100]
 [10100 0]]


# Compare with the Heisenberg layer

In [22]:
Heis=ps.HeisenbergGaussianLayer(N)
nbosonH, nboson2, Dnboson2 = Heis(chir,chii, model)
HModel2 = tf.keras.Model(inputs = xin, outputs=[nbosonH, nboson2 , Dnboson2])

## Run  the model

In [23]:
nbosonH, n2H, Dn2H=HModel2(xtrain) 

### Mean photon number

In [24]:
tf.print(nbosonH)

[[0 100]]


In [25]:
print(np.abs(alpha)**2)

[  0. 100.]


### Mean photon number square

In [26]:
tf.print(n2H)

[[0 10100]]


In [27]:
print(np.abs(alpha)**2+np.abs(alpha)**4)

[    0. 10100.]


### Variance

In [28]:
tf.print(Dn2H)

[[0 100]]


In [29]:
print(np.abs(alpha)**2)

[  0. 100.]
